In [10]:
# GPU 할당 변경하기
GPU_NUM = 1 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
device

device(type='cuda', index=1)

In [13]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from src.dset import DatasetPhysNetED
from src.archs import PhysNetED

import numpy as np
import configparser
import os
import json
import cv2
import matplotlib.pyplot as plt
from skimage import data, io, filters
from tqdm import tqdm

def train_loop(trainloader, model, loss_fn, optimizer):
    model.train()
    size = len(trainloader.dataset)
    train_preds = np.empty(0)
    train_targets = np.empty(0)

    for batch, (inputs, targets) in enumerate(trainloader):
        inputs = inputs.to(device)
        targets = targets.to(device).squeeze()
        pred = model(inputs).squeeze()

        for cnt, pr in enumerate(pred):
            if (batch_size*batch+cnt)%(int(1/overlap)) == 0:
                p = pr.cpu().detach().numpy().squeeze().flatten()
                t = targets[cnt].cpu().detach().numpy().squeeze().flatten()
                train_preds = np.concatenate((train_preds,p))
                train_targets = np.concatenate((train_targets,t))
                
        loss = loss_fn(pred, targets)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1 == 0:
            loss, current = loss.item(), batch * len(targets)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    return train_preds, train_targets
    
def test_loop(testloader, model, loss_fn):
    size = len(testloader.dataset)
    model.eval()    
    test_loss, correct = 0, 0
    test_preds = np.empty(0)
    test_targets = np.empty(0)

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs = inputs.to(device)
            pred = model(inputs).squeeze()
            
            p = pred.cpu().detach().numpy().squeeze().flatten()
            t = targets.cpu().detach().numpy().squeeze().flatten()
            test_preds = np.concatenate((test_preds,p))
            test_targets = np.concatenate((test_targets,t))
        
    return test_preds, test_targets

# if __name__ == '__main__':

configFilename = './config/PURE_PhysNet.cfg'

parser = configparser.ConfigParser(inline_comment_prefixes=('#', ';'))
parser.optionxform = str
parser.read(configFilename)

cfgdict = dict(parser['DEFAULT'].items())
vdir, sigdir = cfgdict['videodataDIR'], cfgdict['signalpath']
print(vdir, sigdir, cfgdict)

# overlap = int(cfgdict['overlap'])
# batch_size = int(cfgdict['batch_size'])
overlap = 0.75
batch_size = 32

print(f'overlap: {overlap}')
print(f'batch_size: {batch_size}')

trainset = DatasetPhysNetED(cfgdict, start = 0, end = 1600, overlap = overlap)
testset = DatasetPhysNetED(cfgdict, start = 1601, end = 2009, overlap = 0)

# Construct DataLoaders
trainloader = DataLoader(trainset,
                        batch_size = batch_size)
testloader = DataLoader(testset,
                        batch_size = batch_size)

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Load Model
model = PhysNetED()
model = model.to(device)

/home/sim/Dataset/rPPG/PURE/01-01/01-01 /home/sim/Dataset/rPPG/PURE/01-01/01-01.json {'dataset': 'PURE', 'model': 'PhysNet', 'videodataDIR': '/home/sim/Dataset/rPPG/PURE/01-01/01-01', 'signalpath': '/home/sim/Dataset/rPPG/PURE/01-01/01-01.json', 'learning_rate': '1e-3', 'depth': '32', 'height': '128', 'width': '128', 'crop': 'True', 'overlap': '0.75', 'batch_size': '32'}
overlap: 0.75
batch_size: 32
Using cuda device


# PhysNet으로 중간 Layer Output 받오기 실습

In [14]:
  
class new_model(nn.Module):
    def __init__(self, model, output_layer = None):
        super().__init__()
        layers = list(model._modules.keys())
        layer_count = 0
        for l in layers:
            if l != output_layer:
                layer_count += 1
            else:
                break
        for i in range(1,len(layers)-layer_count):
            dummy_var = model._modules.pop(layers[-i])
        self.net = nn.Sequential(model._modules)

    def forward(self,x):
        x = self.net(x)
        return x
    
model2 = new_model(model, output_layer = 'loop4')
model2

new_model(
  (net): Sequential(
    (start): Sequential(
      (0): Conv3d(3, 32, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
      (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0)
    )
    (loop1): Sequential(
      (0): AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
      (1): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ELU(alpha=1.0)
      (4): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (5): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ELU(alpha=1.0)
    )
    (encoder): Sequential(
      (0): AvgPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0)
      (1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (2): BatchNorm3d(64, eps=1e-05, m

## 모델의 output shape 확인

In [15]:
for videos, targets in trainloader:
    break
videos = videos.to(device)

videos[:2].shape, videos.device

(torch.Size([2, 3, 32, 128, 128]), device(type='cuda', index=1))

In [16]:
output = model2(videos[:2])
output.shape

torch.Size([2, 64, 8, 8, 8])

## 새로 intermediate까지만 잘라서 만든 모델과 원래 모델의 weight가 같음을 비교

In [17]:
model2.net.start[0].weight

Parameter containing:
tensor([[[[[-0.0389,  0.0876, -0.0110,  0.0522,  0.0716],
           [-0.0253,  0.0301, -0.1119, -0.1016, -0.0985],
           [ 0.1053, -0.0462, -0.1018,  0.0943, -0.0213],
           [ 0.0613, -0.0522, -0.0430,  0.0896, -0.0952],
           [ 0.0101,  0.0962,  0.0626, -0.0846,  0.0962]]],


         [[[ 0.0209,  0.0120,  0.0600, -0.0528, -0.0575],
           [-0.0074, -0.1074, -0.0171,  0.1102, -0.0907],
           [-0.0290, -0.0259,  0.0355, -0.0064,  0.0310],
           [ 0.0687,  0.0885, -0.0729,  0.0871,  0.0904],
           [ 0.0699,  0.0061,  0.1001, -0.0750, -0.0752]]],


         [[[ 0.0776, -0.0485,  0.0534,  0.0239,  0.0136],
           [ 0.1025,  0.0608,  0.0788,  0.0221, -0.0793],
           [ 0.0561, -0.0065, -0.0364,  0.0241, -0.0464],
           [-0.0323, -0.0948,  0.0563,  0.0759, -0.0537],
           [ 0.0707,  0.0077,  0.0702, -0.0670,  0.0605]]]],



        [[[[-0.0512,  0.0503, -0.0665,  0.1148,  0.0939],
           [ 0.0142, -0.1094,  0.062

In [18]:
model.start[0].weight

Parameter containing:
tensor([[[[[-0.0389,  0.0876, -0.0110,  0.0522,  0.0716],
           [-0.0253,  0.0301, -0.1119, -0.1016, -0.0985],
           [ 0.1053, -0.0462, -0.1018,  0.0943, -0.0213],
           [ 0.0613, -0.0522, -0.0430,  0.0896, -0.0952],
           [ 0.0101,  0.0962,  0.0626, -0.0846,  0.0962]]],


         [[[ 0.0209,  0.0120,  0.0600, -0.0528, -0.0575],
           [-0.0074, -0.1074, -0.0171,  0.1102, -0.0907],
           [-0.0290, -0.0259,  0.0355, -0.0064,  0.0310],
           [ 0.0687,  0.0885, -0.0729,  0.0871,  0.0904],
           [ 0.0699,  0.0061,  0.1001, -0.0750, -0.0752]]],


         [[[ 0.0776, -0.0485,  0.0534,  0.0239,  0.0136],
           [ 0.1025,  0.0608,  0.0788,  0.0221, -0.0793],
           [ 0.0561, -0.0065, -0.0364,  0.0241, -0.0464],
           [-0.0323, -0.0948,  0.0563,  0.0759, -0.0537],
           [ 0.0707,  0.0077,  0.0702, -0.0670,  0.0605]]]],



        [[[[-0.0512,  0.0503, -0.0665,  0.1148,  0.0939],
           [ 0.0142, -0.1094,  0.062